In [415]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [434]:
# 유튜브 구독정보 난수생성기로 임의 생성

class RandomGenerator(object):
   
    
    def __init__(self, random_state=1):
        """
        -매개변수-
        random_state : 가중치 초기화를 위한 무작위 난수 생성기의 seed
        """
        self.random_state = random_state
        
    def GeneratingRandomData(self, loc=4, size=50000, scale=3, n_features=10):
        """
        난수생성기로 임의 데이터 생성
        """
    
        np.random.seed(self.random_state)
        # 특정 seed를 가지는 np.random.RandomState() 객체를 만들고, 이 객체를 통해서 난수 생성
        self.data = np.random.normal(loc, scale, size)
        for i in range(len(self.data)):
            self.data[i] = (abs(int(self.data[i])))
                
        return self.data.reshape(-1, n_features)
    
    def ArrayToDataframe(self, random_array, n_sample, feature_names):
        return pd.DataFrame(random_array, index=range(1, n_sample+1), columns=feature_names)

In [435]:
# 난수생성기 객체 생성

dataGenerator = RandomGenerator(156)

In [436]:
randomData = dataGenerator.GeneratingRandomData()
randomData.shape

(5000, 10)

In [438]:
dfRandomData = dataGenerator.ArrayToDataframe(randomData, 5000, feature_names)

In [472]:
dfRandomData.head()

,Game,Cook,Travel,Money,Academy,Learning,Sing,Sports,Machine,Mistery
1,5.0,1.0,1.0,2.0,8.0,3.0,5.0,5.0,1.0,0.0
2,2.0,1.0,3.0,4.0,3.0,2.0,0.0,5.0,0.0,6.0
3,1.0,0.0,2.0,1.0,3.0,9.0,3.0,5.0,11.0,2.0
4,5.0,3.0,4.0,3.0,6.0,1.0,4.0,2.0,4.0,0.0
5,10.0,6.0,3.0,1.0,6.0,3.0,4.0,7.0,2.0,8.0


In [439]:
# feature 이름 리스트 구성

feature_names = ['Game', 'Cook', "Travel", "Money", "Academy", "Learning", "Sing", "Sports", "Machine", "Mistery"]

In [482]:
class Classification(object):
    
    def __init__(self, df_data):
        self.df = df_data
        
    def Extraction(self):
        self.ls = []
        for i in range(1, self.df.shape[0]):
            self.ls.append(self.df.sort_values(by=[i], axis=1).iloc[i, -2:].index.tolist())

In [483]:
cl = Classification(dfRandomData)

In [486]:
cl.ls

[['Sports', 'Academy'],
 ['Sports', 'Mistery'],
 ['Learning', 'Machine'],
 ['Game', 'Academy'],
 ['Mistery', 'Game'],
 ['Academy', 'Travel'],
 ['Travel', 'Sports'],
 ['Machine', 'Mistery'],
 ['Game', 'Academy'],
 ['Machine', 'Academy'],
 ['Game', 'Learning'],
 ['Travel', 'Money'],
 ['Game', 'Travel'],
 ['Sports', 'Learning'],
 ['Learning', 'Sports'],
 ['Money', 'Machine'],
 ['Learning', 'Sing'],
 ['Learning', 'Sing'],
 ['Mistery', 'Sports'],
 ['Learning', 'Sing'],
 ['Academy', 'Machine'],
 ['Mistery', 'Sing'],
 ['Money', 'Academy'],
 ['Mistery', 'Travel'],
 ['Learning', 'Academy'],
 ['Learning', 'Machine'],
 ['Learning', 'Sing'],
 ['Travel', 'Machine'],
 ['Mistery', 'Sports'],
 ['Sing', 'Game'],
 ['Machine', 'Academy'],
 ['Mistery', 'Sports'],
 ['Cook', 'Travel'],
 ['Travel', 'Game'],
 ['Sing', 'Sports'],
 ['Game', 'Sports'],
 ['Cook', 'Mistery'],
 ['Money', 'Machine'],
 ['Cook', 'Money'],
 ['Money', 'Sing'],
 ['Travel', 'Sing'],
 ['Machine', 'Money'],
 ['Game', 'Cook'],
 ['Travel', 'L

In [485]:
cl.Extraction()

In [ ]:
cl.ls

In [ ]:
# 각 샘플에 대해 추출한 가장 선호하는 2개의 특성에 대해 원핫인코딩

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
train_feature = ohe.fit_transform(cl.ls)
train_feature

In [452]:
pd.options.display.max_rows=4000

In [455]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=16)
model.fit(train_feature)

arr = np.array(model.predict(train_feature))

In [459]:
arr.reshape(-1, )

array([4, 2, 1, ..., 1, 9, 4], dtype=int32)